In [1]:
import pandas as pd
import numpy as np

dft = pd.read_csv("times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("complete_distances.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [2]:
def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)
def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

In [3]:
table=pd.read_csv("route.csv", delimiter=";")
time_table=pd.read_csv("route_time.csv", delimiter=";")

In [4]:
import datetime
def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

In [10]:
gained_time1=0
to_2=[]
days = ['martedi', 'mercoledi','giovedì']
#for day in days:
day='martedi'
for i in range(len(time_table[day].dropna())-2):
    try:
        diff=difference(day, i, i+1)
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][i+1]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time1 :
            gained_time1+=(diff-ctime)
            print(f"{day}: primo : {paese} : {gained_time1}")
        else:
            gained_time1+=diff
            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")
            to_2.append(i)
    except:
        print(value)
    #if diff<dft[row[paese].values]:
    #    print("Not acceptable!")

martedi: primo : Cremona, Felice Cavallotti, 2 : 57.98
martedi: primo : Casalpusterlengo : 42.73
martedi: primo : Cremona, Ugolani Dati, 4 : 26.509999999999998
martedi: primo : Crema, Civerchi : 4.009999999999998
martedi: failed : 41.68 : 5.0 : 9.009999999999998
martedi: primo : Sospiro : 11.489999999999998
martedi: failed : 60.6 : 10.0 : 21.49
martedi: primo : Pianengo : 28.009999999999998
martedi: failed : 49.1 : 10.0 : 38.01
martedi: primo : Offanengo : 1.9099999999999966
martedi: failed : 58.87 : 5.0 : 6.909999999999997
martedi: primo : Gussola : 8.159999999999997
martedi: primo : Solarolo : 163.78
martedi: primo : Ostiano : 140.83
martedi: primo : Scandolara Ravara, Italia : 143.53
martedi: primo : Persico Dosimo : 115.21000000000001
martedi: primo : Casalmaggiore : 87.74000000000001
martedi: primo : Gadesco : 73.04
martedi: primo : Viadana : 25.42000000000001
martedi: failed : 42.82 : 15.0 : 40.42000000000001


In [11]:
impossible=[]
gained_time2=0
for i in range(len(to_2)-1):
    try:
        curr=to_2[i]
        diff=difference(day, curr, to_2[i+1])
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][to_2[i+1]]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time2 :
            gained_time2+=(diff-ctime)
            print(f"{day}: secondo : {paese} : {gained_time2}")
        else:
            print(f"{day}: impossible : {paese} : {diff}  {ctime}")
            impossible.append(i)
    except:
        print(f"-----{value}")

martedi: impossible : Sospiro : 15.0  30.4
martedi: impossible : Pianengo : 30.0  41.55
martedi: impossible : Offanengo : 20.0  32.27
martedi: secondo : Viadana : 198.35


In [19]:
#FINAL ALGORITHM
start = dft[get_row(dft, 'Cremona, sesto 39').values]
day='martedi'
#Liste dei due furgoni
f1 = []
f2 = []
#indici
i1 = -1
i2 = -1
#Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
row1 = start
row2 = start
#Lista dei paesi non raggiungibili
impossibile = []
for i in range(len(time_table[day].dropna())):
    paese=table[day][i]
    #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
    value1=row1.loc[:,get_column(dft, paese).values]
    value2=row2.loc[:,get_column(dft, paese).values]
    #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
    if(i1 != -1 and i2 != -1):
        diff1=difference(day, i1, i)
        diff2=difference(day, i2, i)
    else:
        #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
        if(i1 == -1):
            diff1 = value1.values[0][0]
        if(i2 == -1):
            diff2 = value2.values[0][0]
    #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5%) si sceglie il migliore
    #if(float(value1.values[0][0])<= float(diff1) + (float(diff1)/6) or float(value2.values[0][0])<= float(diff2) + float(diff2)/6):
    if(float(value1.values[0][0])<= float(diff1) + 5 or float(value2.values[0][0])<= float(diff2) + 5):
        if(value1.values[0][0] <= value2.values[0][0]):
            f1.append(i)
            i1 = i
            row1 = dft[get_row(dft, table[day][i]).values]
        else:
            f2.append(i)
            i2 = i
            row2 = dft[get_row(dft, table[day][i]).values]
    #Impossibilità di raggiungere la locazione in tempo
    else:
        print(f"current time1 {diff1} time {float(value1.values[0][0])} , current time2 {diff2} time {float(value2.values[0][0])}")
        impossibile.append(i)
print(f1)
print(f2)
print(impossibile)

[0, 3, 5, 6, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[1, 2, 4, 7, 8, 10]
[]


In [57]:
dtime=pd.read_csv("sospirot.csv", delimiter=";", encoding = "ISO-8859-1")
for i in range(len(dtime)):
    for j in range(len(dtime.columns)):
        curr=dtime.iloc[i].iloc[j]
        dtime.iat[i,j]=round(float(curr)/1000)
dtime

,Sospiro
0,61.316667
1,29.150000
2,52.450000
3,6.933333
4,55.116667
...,...
60,63.083333
61,12.916667
62,15.900000
63,8.166667


In [13]:
#time_
table[day][10:13]

10    Offanengo
11      Piadena
12      Gussola
Name: martedi, dtype: object